In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

PATH = Path.cwd().parent.joinpath('data')

In [31]:
vessels = pd.read_csv(PATH.joinpath('processed', 'kse_shadowfleet.csv'))


In [32]:
cols = [x for x in vessels.columns if '-202' in x]

crude = vessels[vessels.commodity == 'crude'].copy()
crude['crude_total'] = crude[cols].sum(axis=1)
crude['months_uninsured'] = crude[cols].isna().sum(axis=1)

products = vessels[vessels.commodity == 'oil products'].copy()
products['products_total'] = products[cols].sum(axis=1)
products['months_uninsured'] = products[cols].isna().sum(axis=1)

crude = crude[['imo', 'vessel_name', 'tanker_size', 'buildyear', 'crude_total', 'months_uninsured']].copy()
products = products[['imo', 'vessel_name', 'tanker_size', 'buildyear', 'products_total', 'months_uninsured']].copy()

vessels = pd.merge(crude, 
                   products, 
                   on=['imo', 'vessel_name', 'tanker_size', 'buildyear'], how='outer')

vessels.months_uninsured_x = vessels.months_uninsured_x.fillna(vessels.months_uninsured_y)
vessels.drop('months_uninsured_y', axis=1, inplace=True)
vessels.rename(columns={'months_uninsured_x': 'months_uninsured'}, inplace=True)
vessels.months_uninsured = vessels.months_uninsured.astype(int)
vessels = pd.merge(vessels,
                   pd.read_csv(PATH.joinpath('processed', 'uninsured.csv')),
                   on='imo', how='left')

vessels.sort_values(by='earliest_sanction_date', inplace=True)
vessels.drop_duplicates(subset='imo', keep='first', inplace=True)
len(vessels)


637

In [33]:
owners = pd.read_csv(PATH.joinpath('processed', 'ownership_changes_end_after_2022.csv'))
len(owners)

3171

In [37]:
roles = ['Registered owner', 'Ship manager / Commercial manager', 'ISM Manager']
col_names = ['ownership', 'shipmanager', 'ism_manager']

for r, n in zip(roles, col_names):

    temp_owners = owners[owners.role==r].groupby('imo').size().reset_index()
    temp_owners.columns = ['imo', f'{n}_changes_after_2022']
    vessels = pd.merge(vessels,
                       temp_owners,
                       on='imo', 
                       how='left')
len(vessels)

637